In [1]:
import numpy as np
import pandas as pd
from transformers import pipeline
from AHBS_AIServices import similarity

# Read the dose sample data

In [2]:
len(pd.read_excel('data/Dose_sample_100.xlsx'))

100

In [62]:
drug_data=pd.read_excel('data/Final_Drug_dose.xlsx',index_col=0)
drug_data

,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,dose_title,dose,dose_range_start,dose_range_end,dose_unit_new,...,maximum_dose,maximum_dose_unit,age_range_start,age_range_end,age_range_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,administration_notes
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
368,dimenhydrinate,Adult,Meniere's Disease (Off-label),NaN,NaN,NaN,NaN,25.0,50.0,mg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25-50 mg PO q8hr for maintenance\n\n50 mg IM f...,NaN
369,dimenhydrinate,Adult,Meniere's Disease (Off-label),NaN,NaN,NaN,50,NaN,NaN,mg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25-50 mg PO q8hr for maintenance\n\n50 mg IM f...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220,vardenafil,Adult,Erectile Dysfunction,NaN,NaN,NaN,10,5.0,20.0,mg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 mg PO ~1 hour before sexual activity; may b...,NaN
20221,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Film-coated tablet,10,5.0,20.0,mg,...,NaN,NaN,NaN,65,years,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,NaN
20222,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Orally disintegrating tablet (ODT),10,NaN,NaN,mg,...,NaN,NaN,NaN,65,years,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,NaN


# classify the medical_condition_h3 col

In [63]:
# Specify the model directory
model_directory = './models/zero_shot_classifier'

# Load the saved pipeline
loaded_zero_shot_classifier_pipeline = pipeline(task='zero-shot-classification', model=model_directory)

In [64]:
candidate_labels= ['medical diagnosis', 'medical procedures or interventions']
drug_data['is_medical_diagnosis'] =drug_data['medical_condition_h3'].apply(lambda x:loaded_zero_shot_classifier_pipeline(x,candidate_labels)['labels'][0]=='medical diagnosis')

In [65]:

drug_data[['medical_condition_h3','is_medical_diagnosis']]

,medical_condition_h3,is_medical_diagnosis
Unnamed: 0,,
367,Prevention of Motion Sickness,True
367,Prevention of Motion Sickness,True
367,Prevention of Motion Sickness,True
368,Meniere's Disease (Off-label),True
369,Meniere's Disease (Off-label),True
...,...,...
20220,Erectile Dysfunction,True
20221,Erectile Dysfunction,True
20222,Erectile Dysfunction,True


In [72]:
drug_data['is_medical_diagnosis'].value_counts()

True     3397
False     635
Name: is_medical_diagnosis, dtype: int64

In [70]:
len(drug_data)

4032

# Read the ICD10 Codes data

In [66]:
df_codes=pd.read_csv('data/idc10_top_to_leaf_data.csv')
df_codes

,diseaseCode,description
0,A00.0,"certain infectious and parasitic diseases , in..."
1,A00.1,"certain infectious and parasitic diseases , in..."
2,A00.9,"certain infectious and parasitic diseases , in..."
3,A01.00,"certain infectious and parasitic diseases , in..."
4,A01.01,"certain infectious and parasitic diseases , in..."
...,...,...
50123,Z99.12,factors influencing health status and contact ...
50124,Z99.2,factors influencing health status and contact ...
50125,Z99.3,factors influencing health status and contact ...
50126,Z99.81,factors influencing health status and contact ...


In [67]:
df_codes[df_codes.duplicated(['description'])]

,diseaseCode,description
41487,U49,"codes for special purposes , provisional assig..."
41488,U50,"codes for special purposes , provisional assig..."
41489,U85,"codes for special purposes , provisional assig..."


In [68]:
# disease_code_dict=dict(zip(df_codes['description'].tolist(),df_codes['diseaseCode'].tolist()))

# Compute the similarity between the two sources

In [8]:
# similarity_df=pd.DataFrame(similarity.get_sentences_similarity(sentence_list_1=drug_data['medical_condition_h3'],sentence_list_2=df_codes['description'].tolist(),get_score=True),columns=['medical_condition_h3','most_similar_description','similarity_score'])

In [73]:
similarities=similarity.get_sentences_similarity(sentence_list_1=drug_data['medical_condition_h3'].unique().tolist(),sentence_list_2=df_codes['description'].tolist(),get_score=True)

In [74]:
similarity_df=pd.DataFrame(similarities,columns=['medical_condition_h3','most_similar_description','similarity_score'])
len(similarity_df)

810

In [75]:
sim_dict=dict(zip(similarity_df['medical_condition_h3'].tolist(),similarity_df['most_similar_description'].tolist()))
sim_dict

{'Prevention of Motion Sickness': 'injury, poisoning and certain other consequences of external causes , other and unspecified effects of external causes , other and unspecified effects of other external causes , motion sickness , sequela',
 "Meniere's Disease (Off-label)": 'diseases of the ear and mastoid process , diseases of inner ear , disorders of vestibular function , ménières disease , bilateral',
 'Allergic Rhinitis': 'diseases of the respiratory system , other diseases of upper respiratory tract , vasomotor and allergic rhinitis , allergic rhinitis, unspecified',
 'Urticaria': 'diseases of the skin and subcutaneous tissue , urticaria and erythema , urticaria , idiopathic urticaria',
 'Allergic Rhinitis & Urticaria': 'diseases of the skin and subcutaneous tissue , urticaria and erythema , urticaria , allergic urticaria',
 'Chronic Idiopathic Urticaria': 'diseases of the skin and subcutaneous tissue , urticaria and erythema , urticaria , idiopathic urticaria',
 'Perennial Allerg

In [76]:
sim_score_dict=dict(zip(similarity_df['medical_condition_h3'].tolist(),similarity_df['similarity_score'].tolist()))
sim_score_dict

{'Prevention of Motion Sickness': 0.7142633487102691,
 "Meniere's Disease (Off-label)": 0.7455485167840152,
 'Allergic Rhinitis': 0.7060830123913676,
 'Urticaria': 0.7652609552238315,
 'Allergic Rhinitis & Urticaria': 0.7625888645037733,
 'Chronic Idiopathic Urticaria': 0.7850641636257886,
 'Perennial Allergic Rhinitis, Chronic Idiopathic Urticaria': 0.64111050893206,
 'Cough': 0.44279053245680144,
 'Anesthesia': 0.6338745664687833,
 'MAC Sedation': 0.5396731050115812,
 'Postoperative Nausea/Vomiting': 0.5605190528150404,
 'ICU Patient': 0.3538045626142002,
 'Anesthesia Induction': 0.5814224736110204,
 'Resistant Depression (Off-label)': 0.3728966928394403,
 'Sedation/Analgesia (Off-label)': 0.46808081568982757,
 'Subarachnoid block': 0.337813788306733,
 'Local Anesthesia': 0.5951740119031848,
 'Regional Anesthesia': 0.5492820669819045,
 'Postsurgical Analgesia for Arthroscopic Subacromial Decompression': 0.4190066668184466,
 'Regional Anesthesia ': 0.5492820669819045,
 'Surgery Premed

In [77]:
similarities

[['Prevention of Motion Sickness',
  'injury, poisoning and certain other consequences of external causes , other and unspecified effects of external causes , other and unspecified effects of other external causes , motion sickness , sequela',
  0.7142633487102691],
 ["Meniere's Disease (Off-label)",
  'diseases of the ear and mastoid process , diseases of inner ear , disorders of vestibular function , ménières disease , bilateral',
  0.7455485167840152],
 ['Allergic Rhinitis',
  'diseases of the respiratory system , other diseases of upper respiratory tract , vasomotor and allergic rhinitis , allergic rhinitis, unspecified',
  0.7060830123913676],
 ['Urticaria',
  'diseases of the skin and subcutaneous tissue , urticaria and erythema , urticaria , idiopathic urticaria',
  0.7652609552238315],
 ['Allergic Rhinitis & Urticaria',
  'diseases of the skin and subcutaneous tissue , urticaria and erythema , urticaria , allergic urticaria',
  0.7625888645037733],
 ['Chronic Idiopathic Urticar

In [78]:
drug_data['most_similar_description']=drug_data['medical_condition_h3'].apply(lambda x:sim_dict[x])
drug_data['similarity_score']=drug_data['medical_condition_h3'].apply(lambda x:sim_score_dict[x])

In [79]:
drug_data

,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,dose_title,dose,dose_range_start,dose_range_end,dose_unit_new,...,age_range_end,age_range_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,administration_notes,is_medical_diagnosis,most_similar_description,similarity_score
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN,True,"injury, poisoning and certain other consequenc...",0.714263
367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN,True,"injury, poisoning and certain other consequenc...",0.714263
367,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN,True,"injury, poisoning and certain other consequenc...",0.714263
368,dimenhydrinate,Adult,Meniere's Disease (Off-label),NaN,NaN,NaN,NaN,25.0,50.0,mg,...,NaN,NaN,NaN,NaN,NaN,25-50 mg PO q8hr for maintenance\n\n50 mg IM f...,NaN,True,"diseases of the ear and mastoid process , dise...",0.745549
369,dimenhydrinate,Adult,Meniere's Disease (Off-label),NaN,NaN,NaN,50,NaN,NaN,mg,...,NaN,NaN,NaN,NaN,NaN,25-50 mg PO q8hr for maintenance\n\n50 mg IM f...,NaN,True,"diseases of the ear and mastoid process , dise...",0.745549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220,vardenafil,Adult,Erectile Dysfunction,NaN,NaN,NaN,10,5.0,20.0,mg,...,NaN,NaN,NaN,NaN,NaN,10 mg PO ~1 hour before sexual activity; may b...,NaN,True,"diseases of the genitourinary system , disease...",0.690745
20221,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Film-coated tablet,10,5.0,20.0,mg,...,65,years,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,NaN,True,"diseases of the genitourinary system , disease...",0.690745
20222,vardenafil,Geriatric,Erectile Dysfunction,NaN,NaN,Orally disintegrating tablet (ODT),10,NaN,NaN,mg,...,65,years,NaN,NaN,NaN,<65 years\n\nFilm-coated tablet: 10 mg PO ~1 h...,NaN,True,"diseases of the genitourinary system , disease...",0.690745


In [82]:
drug_data[['medical_condition_h3','most_similar_description']]

,medical_condition_h3,most_similar_description
Unnamed: 0,,
367,Prevention of Motion Sickness,"injury, poisoning and certain other consequenc..."
367,Prevention of Motion Sickness,"injury, poisoning and certain other consequenc..."
367,Prevention of Motion Sickness,"injury, poisoning and certain other consequenc..."
368,Meniere's Disease (Off-label),"diseases of the ear and mastoid process , dise..."
369,Meniere's Disease (Off-label),"diseases of the ear and mastoid process , dise..."
...,...,...
20220,Erectile Dysfunction,"diseases of the genitourinary system , disease..."
20221,Erectile Dysfunction,"diseases of the genitourinary system , disease..."
20222,Erectile Dysfunction,"diseases of the genitourinary system , disease..."


In [83]:
drug_data.columns

Index(['drug_name', 'Age Range', 'medical_condition_h3',
       'medical_condition_h4', 'medical_condition_h5', 'dose_title', 'dose',
       'dose_range_start', 'dose_range_end', 'dose_unit_new', 'per_with_unit',
       'is_divided', 'route_of_administration', 'PRN', 'infusion_rate_unit',
       'frequency', 'frequency_range_start', 'frequency_range_end',
       'frequency_unit', 'duration', 'duration_unit', 'maximum_dose',
       'maximum_dose_unit', 'age_range_start', 'age_range_end',
       'age_range_unit', 'body_weight_range_start', 'body_weight_range_end',
       'body_weight_range_unit', 'reference', 'administration_notes',
       'is_medical_diagnosis', 'most_similar_description', 'similarity_score'],
      dtype='object')

In [91]:
df_codes

,diseaseCode,description
0,A00.0,"certain infectious and parasitic diseases , in..."
1,A00.1,"certain infectious and parasitic diseases , in..."
2,A00.9,"certain infectious and parasitic diseases , in..."
3,A01.00,"certain infectious and parasitic diseases , in..."
4,A01.01,"certain infectious and parasitic diseases , in..."
...,...,...
50123,Z99.12,factors influencing health status and contact ...
50124,Z99.2,factors influencing health status and contact ...
50125,Z99.3,factors influencing health status and contact ...
50126,Z99.81,factors influencing health status and contact ...


In [95]:
drug_data_merged=drug_data.merge(df_codes,left_on='most_similar_description',right_on='description')


In [99]:
drug_data_merged

,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,dose_title,dose,dose_range_start,dose_range_end,dose_unit_new,...,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,administration_notes,is_medical_diagnosis,most_similar_description,similarity_score,diseaseCode,description
0,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,"injury, poisoning and certain other consequenc..."
1,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,"injury, poisoning and certain other consequenc..."
2,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,NaN,NaN,50.0,100.0,mg,...,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,"injury, poisoning and certain other consequenc..."
3,dimenhydrinate,Pediatric,Prevention of Motion Sickness,NaN,NaN,General dose,37.5,NaN,NaN,mg,...,NaN,NaN,NaN,General dose: 1.25 mg/kg or 37.5 mg/sq.meter I...,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,"injury, poisoning and certain other consequenc..."
4,dimenhydrinate,Pediatric,Prevention of Motion Sickness,NaN,NaN,General dose,37.5,NaN,NaN,mg,...,NaN,NaN,NaN,General dose: 1.25 mg/kg or 37.5 mg/sq.meter I...,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,"injury, poisoning and certain other consequenc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,tropicamide,Pediatric,Refraction (Mydriasis),NaN,NaN,Adult,NaN,NaN,NaN,drop,...,NaN,NaN,NaN,"As adult; 1-2 gtt of 1% solution in eye(s), re...",NaN,False,"diseases of the eye and adnexa , disorders of ...",0.585913,H52.13,"diseases of the eye and adnexa , disorders of ..."
4028,tropicamide,Adult,Funduscopic Exam (Cycloplegia),NaN,NaN,NaN,NaN,1.0,2.0,drop,...,NaN,NaN,NaN,1-2 gtt of 0.5% solution in eyes 15-20 min bef...,NaN,True,"diseases of the eye and adnexa , disorders of ...",0.456150,H50.412,"diseases of the eye and adnexa , disorders of ..."
4029,tropicamide,Pediatric,Funduscopic Exam (Cycloplegia),NaN,NaN,Adult dose,NaN,1.0,2.0,drop,...,NaN,NaN,NaN,As adult; 1-2 gtt of 0.5% solution in eyes 15-...,NaN,True,"diseases of the eye and adnexa , disorders of ...",0.456150,H50.412,"diseases of the eye and adnexa , disorders of ..."
4030,finasteride,Adult,Androgenic Alopecia (Men Only),NaN,NaN,Propecia,1,NaN,NaN,mg,...,NaN,NaN,NaN,Propecia: 1 mg PO qDay for at least 3 months,NaN,True,"diseases of the skin and subcutaneous tissue ,...",0.690526,L64.8,"diseases of the skin and subcutaneous tissue ,..."


In [100]:
drug_data_merged=drug_data_merged[['drug_name', 'Age Range', 'medical_condition_h3',
       'medical_condition_h4', 'medical_condition_h5','is_medical_diagnosis', 'most_similar_description', 'similarity_score','diseaseCode','dose_title', 'dose',
       'dose_range_start', 'dose_range_end', 'dose_unit_new', 'per_with_unit',
       'is_divided', 'route_of_administration', 'PRN', 'infusion_rate_unit',
       'frequency', 'frequency_range_start', 'frequency_range_end',
       'frequency_unit', 'duration', 'duration_unit', 'maximum_dose',
       'maximum_dose_unit', 'age_range_start', 'age_range_end',
       'age_range_unit', 'body_weight_range_start', 'body_weight_range_end',
       'body_weight_range_unit', 'reference', 'administration_notes',
       ]]

In [101]:
drug_data_merged

,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,is_medical_diagnosis,most_similar_description,similarity_score,diseaseCode,dose_title,...,maximum_dose,maximum_dose_unit,age_range_start,age_range_end,age_range_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,administration_notes
0,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,NaN,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
1,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,NaN,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
2,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,NaN,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
3,dimenhydrinate,Pediatric,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,General dose,...,300,mg,NaN,NaN,NaN,NaN,NaN,NaN,General dose: 1.25 mg/kg or 37.5 mg/sq.meter I...,NaN
4,dimenhydrinate,Pediatric,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,General dose,...,300,mg,NaN,NaN,NaN,NaN,NaN,NaN,General dose: 1.25 mg/kg or 37.5 mg/sq.meter I...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,tropicamide,Pediatric,Refraction (Mydriasis),NaN,NaN,False,"diseases of the eye and adnexa , disorders of ...",0.585913,H52.13,Adult,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"As adult; 1-2 gtt of 1% solution in eye(s), re...",NaN
4028,tropicamide,Adult,Funduscopic Exam (Cycloplegia),NaN,NaN,True,"diseases of the eye and adnexa , disorders of ...",0.456150,H50.412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-2 gtt of 0.5% solution in eyes 15-20 min bef...,NaN
4029,tropicamide,Pediatric,Funduscopic Exam (Cycloplegia),NaN,NaN,True,"diseases of the eye and adnexa , disorders of ...",0.456150,H50.412,Adult dose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,As adult; 1-2 gtt of 0.5% solution in eyes 15-...,NaN
4030,finasteride,Adult,Androgenic Alopecia (Men Only),NaN,NaN,True,"diseases of the skin and subcutaneous tissue ,...",0.690526,L64.8,Propecia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Propecia: 1 mg PO qDay for at least 3 months,NaN


In [102]:
# def replace_columns_with_none(row, columns_to_replace):
#     if not row['is_medical_diagnosis']:
#         return [None] * len(columns_to_replace)
#     else:
#         return [row[column] for column in columns_to_replace]
# 
# # Specify the list of columns to replace with None
# columns_to_replace = ['diseaseCode', 'similarity_score', 'most_similar_diagnosis']
def replace_columns_with_none(row, columns_to_replace):
    if not row['is_medical_diagnosis']:
        return [np.NaN] * len(columns_to_replace)
    else:
        return [row[column] for column in columns_to_replace]

# Specify the list of columns to replace with None
columns_to_replace = ['diseaseCode', 'similarity_score', 'most_similar_description']
# Apply the function to the specified columns
drug_data_merged[columns_to_replace] = drug_data_merged.apply(
    lambda row: replace_columns_with_none(row, columns_to_replace),
    axis=1,
    result_type='expand'
)

C:\Users\omar.wafy\AppData\Local\Temp\ipykernel_9432\1159961089.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_data_merged[columns_to_replace] = drug_data_merged.apply(


In [103]:
drug_data_merged

,drug_name,Age Range,medical_condition_h3,medical_condition_h4,medical_condition_h5,is_medical_diagnosis,most_similar_description,similarity_score,diseaseCode,dose_title,...,maximum_dose,maximum_dose_unit,age_range_start,age_range_end,age_range_unit,body_weight_range_start,body_weight_range_end,body_weight_range_unit,reference,administration_notes
0,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,NaN,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
1,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,NaN,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
2,dimenhydrinate,Adult,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,NaN,...,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"50-100 mg PO/IV/IM q4-6hr PRN, 30 minutes befo...",NaN
3,dimenhydrinate,Pediatric,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,General dose,...,300,mg,NaN,NaN,NaN,NaN,NaN,NaN,General dose: 1.25 mg/kg or 37.5 mg/sq.meter I...,NaN
4,dimenhydrinate,Pediatric,Prevention of Motion Sickness,NaN,NaN,True,"injury, poisoning and certain other consequenc...",0.714263,T75.3XXS,General dose,...,300,mg,NaN,NaN,NaN,NaN,NaN,NaN,General dose: 1.25 mg/kg or 37.5 mg/sq.meter I...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,tropicamide,Pediatric,Refraction (Mydriasis),NaN,NaN,False,NaN,NaN,NaN,Adult,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"As adult; 1-2 gtt of 1% solution in eye(s), re...",NaN
4028,tropicamide,Adult,Funduscopic Exam (Cycloplegia),NaN,NaN,True,"diseases of the eye and adnexa , disorders of ...",0.456150,H50.412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-2 gtt of 0.5% solution in eyes 15-20 min bef...,NaN
4029,tropicamide,Pediatric,Funduscopic Exam (Cycloplegia),NaN,NaN,True,"diseases of the eye and adnexa , disorders of ...",0.456150,H50.412,Adult dose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,As adult; 1-2 gtt of 0.5% solution in eyes 15-...,NaN
4030,finasteride,Adult,Androgenic Alopecia (Men Only),NaN,NaN,True,"diseases of the skin and subcutaneous tissue ,...",0.690526,L64.8,Propecia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Propecia: 1 mg PO qDay for at least 3 months,NaN


In [104]:
len(drug_data_merged)

4032

In [127]:
drug_data_merged.to_excel('data/Drug_Dose_icd10.xlsx',index=False)

# Testing

In [105]:
threshold=0.6
high_conf_data=drug_data_merged[drug_data_merged['similarity_score'] >=threshold].copy()
low_conf_data=drug_data_merged[drug_data_merged['similarity_score'] <threshold].copy()

In [106]:
len(low_conf_data)

995

In [107]:
len(high_conf_data)

2402

In [36]:
sample_high_conf=high_conf_data.sample(10,random_state=0)[['medical_condition_h3','is_medical_diagnosis','diseaseCode']].copy()
sample_low_conf=low_conf_data.sample(10,random_state=0)[['medical_condition_h3','is_medical_diagnosis','diseaseCode']].copy()

In [112]:
drug_data_merged['medical_condition_h3']=drug_data_merged['medical_condition_h3'].str.strip()

C:\Users\omar.wafy\AppData\Local\Temp\ipykernel_9432\3046753818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_data_merged['medical_condition_h3']=drug_data_merged['medical_condition_h3'].str.strip()


In [37]:
sample_high_conf['medical_condition_h3']=sample_high_conf['medical_condition_h3'].str.strip()
sample_low_conf['medical_condition_h3']=sample_low_conf['medical_condition_h3'].str.strip()

In [38]:
gpt_high_conf_suggest=pd.DataFrame( {
    'Condition': [
        'Benign Prostatic Hyperplasia (BPH)',
        'HIV Infection',
        'Skin and Skin Structure Infections',
        'Anthrax Infection',
        'Schistosomiasis',
        'Primary Generalized Tonic-Clonic Seizures',
        'Primary Generalized Tonic-Clonic Seizures',
        'Major Depressive Disorder',
        'Hypokalemia',
        'Cryptococcal Meningitis'
    ],
    'Medical Condition': [
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes'
    ],
    'Suggested ICD-10 Code': [
        'N40',
        'B20',
        'L00-L99',
        'A22',
        'B65-B83',
        'G40.2',
        'G40.2',
        'F32',
        'E87.6',
        'B45.1'
    ]
})

In [117]:
gpt_high_conf_suggest.merge(drug_data_merged[['medical_condition_h3','diseaseCode']], how='inner', left_on='Condition',right_on='medical_condition_h3').drop_duplicates()

,Condition,Medical Condition,Suggested ICD-10 Code,medical_condition_h3,diseaseCode
0,Benign Prostatic Hyperplasia (BPH),Yes,N40,Benign Prostatic Hyperplasia (BPH),N40.1
1,HIV Infection,Yes,B20,HIV Infection,B20
15,Skin and Skin Structure Infections,Yes,L00-L99,Skin and Skin Structure Infections,B00.1
20,Anthrax Infection,Yes,A22,Anthrax Infection,A22.0
22,Schistosomiasis,Yes,B65-B83,Schistosomiasis,B65.1
24,Primary Generalized Tonic-Clonic Seizures,Yes,G40.2,Primary Generalized Tonic-Clonic Seizures,G40.411
100,Major Depressive Disorder,Yes,F32,Major Depressive Disorder,F33.9
118,Hypokalemia,Yes,E87.6,Hypokalemia,E87.6
138,Cryptococcal Meningitis,Yes,B45.1,Cryptococcal Meningitis,B45.2


In [118]:
len(sample_high_conf)

10

In [119]:
sample_high_conf

,medical_condition_h3,is_medical_diagnosis,diseaseCode
44,Otitis Media,True,H66.90
56,Migraine Headache,True,G43.911
99,Cryptococcal Meningitis,True,B45.2
47,Heart Failure,True,I50.9
25,HIV Infection,True,B20
6,Benign Prostatic Hyperplasia (BPH),True,N40.1
54,Major Depressive Disorder,True,F33.9
98,Type 1 or 2 Diabetes Mellitus,True,E10.8
66,"GERD, Erosive Esophagitis",True,K20.0
40,Malaria,True,B54


In [120]:
sample_low_conf=low_conf_data[low_conf_data['is_medical_diagnosis']==True].sample(10,random_state=0)[['medical_condition_h3','is_medical_diagnosis','diseaseCode']].copy()

In [121]:
# test on low conf
sample_low_conf['medical_condition_h3'].tolist()

['Left Ventricular Dysfunction After Myocardial Infarction',
 'Dermatofibrosarcoma Protuberans',
 'Depression',
 'Anxiety',
 "Traveler's Diarrhea",
 'Acute Pulmonary Edema/Hypertensive Crisis/Increased Intracranial Pressure',
 'Cancers (Off-label)',
 'Partial Seizures (Monotherapy)',
 'Visipaque 320 ',
 'GERD']

In [44]:
spec_low_conf_data=low_conf_data[low_conf_data['is_medical_diagnosis']==True]

In [124]:
gpt_low_conf_suggest=pd.DataFrame( {
    'Condition': [
        'Partial-Onset Seizures (Conversion to Monotherapy)',
        'Influenza A and B Treatment',
        'Soft Tissue Infections',
        'Hot flashes-cancer related (Off-label)',
        'Lennox-Gastaut Syndrome',
        'Cough',
        'Patent Ductus Arteriosus',
        'Uveitis',
        'Influenza A and B Prophylaxis',
        'Intra-abdominal Infections'
    ],
    'Medical Condition': [
        'Yes',
        'Yes',
        'Yes',
        'No',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes',
        'Yes'
    ],
    'Suggested ICD-10 Code': [
        'G40.5',
        'J10-J11',
        'L00-L99',
        'Not Applicable',
        'G40.4',
        'R05',
        'PDA-related condition',
        'H20-H22',
        'Z23',
        'K65-K67'
    ]
}
)

In [126]:
gpt_low_conf_suggest.merge(drug_data_merged[['medical_condition_h3','is_medical_diagnosis','diseaseCode']], how='inner', left_on='Condition',right_on='medical_condition_h3').drop_duplicates()

,Condition,Medical Condition,Suggested ICD-10 Code,medical_condition_h3,is_medical_diagnosis,diseaseCode
0,Partial-Onset Seizures (Conversion to Monother...,Yes,G40.5,Partial-Onset Seizures (Conversion to Monother...,True,G40.101
40,Influenza A and B Treatment,Yes,J10-J11,Influenza A and B Treatment,True,J15.3
47,Soft Tissue Infections,Yes,L00-L99,Soft Tissue Infections,True,B09
51,Hot flashes-cancer related (Off-label),No,Not Applicable,Hot flashes-cancer related (Off-label),True,X10.0XXS
52,Lennox-Gastaut Syndrome,Yes,G40.4,Lennox-Gastaut Syndrome,True,G40.813
64,Cough,Yes,R05,Cough,True,R05.9
85,Patent Ductus Arteriosus,Yes,PDA-related condition,Patent Ductus Arteriosus,True,Q25.0
86,Uveitis,Yes,H20-H22,Uveitis,True,L56.9
92,Influenza A and B Prophylaxis,Yes,Z23,Influenza A and B Prophylaxis,True,J18.2
98,Intra-abdominal Infections,Yes,K65-K67,Intra-abdominal Infections,True,A04.9
